インポート

In [17]:
import folium
from folium import plugins
import geopandas as gpd
import pandas as pd
import contextily as ctx
import plotly.express as px
import contextily as cx
import numpy as np
import matplotlib.pyplot as plt


# set font
import matplotlib as mpl

# for Mac
# mpl.rc('font',family='Hiragino Maru Gothic Pro')

# for PC
mpl.rc('font',family='MS Gothic')


データの読み込み

In [18]:
data1 = pd.read_csv('data/COVID-custom8-2.csv')
data2 = pd.read_csv('data/airport.csv')

データの確認

In [19]:
data1

,ID,age,Gender,longitude,latitude,infection,Gender.1,month,day
0,1,30,Male,139.642347,35.447504,2020/1/15,1,1,15
1,2,40,Male,116.409685,39.903832,2020/1/24,1,1,24
2,3,30,Female,116.409685,39.903832,2020/1/25,0,1,25
3,4,40,Male,116.409685,39.903832,2020/1/26,1,1,26
4,5,40,Male,116.409685,39.903832,2020/1/28,1,1,28
...,...,...,...,...,...,...,...,...,...
124874,133170,30,Male,138.412868,36.744750,2020/11/30,1,11,30
124875,133171,70,Female,138.440643,36.922904,2020/11/30,0,11,30
124876,133172,70,Female,137.851094,36.503017,2020/11/30,0,11,30
124877,133173,50,Male,137.851094,36.503017,2020/11/30,1,11,30


In [20]:
data2

,name,tag,latitude,longitude
0,調布飛行場,1,35.672279,139.528804
1,羽田空港,1,35.549563,139.779828
2,成田国際空港,1,35.772465,140.392790
3,茨城空港,1,36.178137,140.407525
4,福島空港,1,37.228600,140.428342
5,仙台国際空港,1,38.137901,140.929725
6,山形空港,1,38.410668,140.366665
7,新潟空港,1,37.952693,139.113310
8,庄内空港,1,38.815994,139.787621
9,いわて花巻空港,1,39.421567,141.138581


緯度と経度の平均

In [21]:
# get center lat/lon
center_lat = data1['latitude'].mean()
center_lon = data1['longitude'].mean()

print(center_lat,center_lon)

35.24261591247848 137.59804835384338


In [ ]:
# 35.689294093556576, 139.691859387526

データ区分

In [22]:
male = data1[data1['Gender']=='Male']
female = data1[data1['Gender']=='Female']

m1 = data1[data1['month']==1]
m2 = data1[data1['month']==2]
m3 = data1[data1['month']==3]
m4 = data1[data1['month']==4]
m5 = data1[data1['month']==5]
m6 = data1[data1['month']==6]
m7 = data1[data1['month']==7]
m8 = data1[data1['month']==8]
m9 = data1[data1['month']==9]
m10 = data1[data1['month']==10]
m11 = data1[data1['month']==11]
m12 = data1[data1['month']==12]

ap = data2[data2['tag']==1]

Wshin_lat = 35.689294093556576
Wshin_lon = 139.691859387526


マップの確認

In [28]:
# make the map
# m = folium.Map(location=[center_lat,center_lon], zoom_start=8)

#地図の作成
m = folium.Map(location=[Wshin_lat,Wshin_lon], zoom_start=13)

#クラスターの定義
group = plugins.MarkerCluster().add_to(m)

# df をループしてマーカーを作る
for index, row in m1.iterrows():
    latlon = [row['latitude'],row['longitude']]
    folium.Marker(latlon, 
                  popup='<div style="width:350px">'+row['infection']+'</div>', 
                  tooltip=row['infection'],
                  
                ).add_to(group)
    
for index, row in m2.iterrows():
    latlon = [row['latitude'],row['longitude']]
    folium.Marker(latlon, 
                  popup='<div style="width:350px">'+row['infection']+'</div>', 
                  tooltip=row['infection'],
                  
                ).add_to(group)

for index, row in m3.iterrows():
    latlon = [row['latitude'],row['longitude']]
    folium.Marker(latlon, 
                  popup='<div style="width:350px">'+row['infection']+'</div>', 
                  tooltip=row['infection'],
                  
                ).add_to(group)
    
for index, row in ap.iterrows():
    latlon = [row['latitude'],row['longitude']]
    folium.Marker(latlon, 
                  popup='<div style="width:350px">'+row['name']+'</div>', 
                  tooltip=row['name'],
                  icon=folium.Icon(color='red')
                ).add_to(m)


# show the map
m

In [25]:
# make the map
# m = folium.Map(location=[center_lat,center_lon], zoom_start=8)

#地図の作成
m = folium.Map(location=[Wshin_lat,Wshin_lon], zoom_start=13)

#クラスターの定義
# group = plugins.MarkerCluster().add_to(m)

# df をループしてマーカーを作る
for index, row in ap.iterrows():
    latlon = [row['latitude'],row['longitude']]
    folium.Marker(latlon, 
                  popup='<div style="width:350px">'+row['name']+'</div>', 
                  tooltip=row['name'],
                  
                ).add_to(m)
    


# show the map
m

感想
感染症のハザードマップを作ろうとしたが、データの地点が役所なので、どこの地区が感染者が多いかというマップになりそう。
月別での感染者数を比べて、最初に感染対策をする地区を提案できるかもしれない。